## Unit economics online channel

In [1]:
print('Doing the math, wait a minute')

Doing the math, wait a minute


In [2]:
import pandas as pd

In [4]:
directory = 'p:/FD-Supermarket/Отчетность/Расчет PL Перекресток Быстро/2023/'
data_ue_pnl, data_ue_delivery_cost = 'data_ue_pnl.xlsx', 'data_ue_delivery_cost.xlsx'
data_ue_pnl, data_ue_delivery_cost = pd.read_excel(
    directory+data_ue_pnl, header=0), pd.read_excel(directory+data_ue_delivery_cost, header=0)

In [5]:
#prepare data of delivery cost (group the duplicated cities and process numbers)
data_ue_delivery_cost_2 = data_ue_delivery_cost.groupby(['Город', 'Месяц'], as_index=False).agg({'Факт TY':'sum'})
data_ue_delivery_cost_2['Факт TY'] = data_ue_delivery_cost_2['Факт TY'] / -1000

In [6]:
#split table to stores and others objects
#only retail stores, #excclude log.cost becasue it will be taken from other source
data_ue_pnl_stores = data_ue_pnl.loc[(data_ue_pnl['Тип ЦФО'] == 'Магазин') & (
    data_ue_pnl['Даркстор'] == 'Магазин') & (data_ue_pnl['Статья FBR'] != 'Расходы на доставку')]
#group by city, #add column with cost type
data_ue_pnl_stores_2 = data_ue_pnl_stores.groupby(
    ['Город', 'Месяц', 'Статья FBR'], as_index=False).agg({'Факт TY':'sum'})
data_ue_pnl_stores_2['Вид расходов'], data_ue_pnl_stores_2['СМ/DS'] = 'Прямые', 'СМ'


#only darktstores
data_ue_pnl_darkstores = data_ue_pnl.loc[data_ue_pnl['Даркстор'] == 'Даркстор']
#group by city, #add column with cost type
data_ue_pnl_darkstores_2 = data_ue_pnl_darkstores.groupby(
    ['Город', 'Месяц', 'Статья FBR'], as_index=False).agg({'Факт TY':'sum'})
data_ue_pnl_darkstores_2['Вид расходов'], data_ue_pnl_darkstores_2['СМ/DS'] = 'Прямые', 'DS'

#only others (administrative objects), #excclude log.cost becasue it will be taken from other source
data_ue_pnl_others = data_ue_pnl.loc[(data_ue_pnl['Тип ЦФО'] != 'Магазин') & (
    data_ue_pnl['Статья FBR'] != 'Расходы на доставку')]

In [7]:
#create list of columns other than exceptions
attribute_columns = list(data_ue_pnl_others.columns.difference(['Факт TY', 'Бюджет TY', 'Факт LY']))

#### Drivers for general retail cost allocations:
- total number of orders

In [8]:
#calculate the number of orders by cities
orders = data_ue_pnl_stores.loc[data_ue_pnl_stores['Тип расходов']=='Кол-во заказов'].groupby(
    ['Город', 'Месяц', 'Дивизион', 'Макро'], as_index=False).agg({'Факт TY':'sum'})

#calculate the number of orders by division
orders_div = orders.groupby(
    ['Месяц', 'Дивизион'], as_index=False).agg({'Факт TY':'sum'}).rename(
    columns = {'Факт TY':'Факт TY_div'})

#calculate the number of orders by macro-division
orders_macro = orders.groupby(
    ['Месяц', 'Макро'], as_index=False).agg({'Факт TY':'sum'}).rename(
    columns = {'Факт TY':'Факт TY_macro'})

#calculate the number of orders by total (month)
orders_total = orders.groupby(
    ['Месяц'], as_index=False).agg({'Факт TY':'sum'}).rename(
    columns = {'Факт TY':'Факт TY_total'})

In [9]:
#add grouped number of orders into the main table
orders_2 = orders.merge(orders_div, on=['Месяц', 'Дивизион'], how='left')
orders_2 = orders_2.merge(orders_macro, on=['Месяц', 'Макро'], how='left')
orders_2 = orders_2.merge(orders_total, on=['Месяц'], how='left')

In [10]:
#calculate the share of the city in different levels
orders_2['div_share'] = orders_2['Факт TY'] / orders_2['Факт TY_div']
orders_2['macro_share'] = orders_2['Факт TY'] / orders_2['Факт TY_macro']
orders_2['total_share'] = orders_2['Факт TY'] / orders_2['Факт TY_total']

In [11]:
#create tables which includes all 3 levels of allocations
cost_division = data_ue_pnl_others.loc[data_ue_pnl_others['Город']=='Аллокации Дивизионы'].groupby(
    ['Месяц', 'Дивизион', 'Статья FBR'], as_index=False).agg({'Факт TY':'sum'})
cost_macro = data_ue_pnl_others.loc[data_ue_pnl_others['Город']=='Аллокации Макро'].groupby(
    ['Месяц', 'Макро', 'Статья FBR'], as_index=False).agg({'Факт TY':'sum'})
cost_total = data_ue_pnl_others.loc[data_ue_pnl_others['Город']=='Аллокации ТС'].groupby(
    ['Месяц', 'Статья FBR'], as_index=False).agg({'Факт TY':'sum'})

In [12]:
#create a text key
orders_2['key'] = orders_2['Город']+orders_2['Месяц'].astype('str')+orders_2[
    'Дивизион']+orders_2['Макро']

In [13]:
#create a Cartesian product
key_list = orders_2['key'].unique()
accounts_list = data_ue_pnl_others['Статья FBR'].unique()

cartesian_df = pd.MultiIndex.from_product([key_list, accounts_list]).to_frame().reset_index(drop=True)
cartesian_df.columns = ['key', 'Статья FBR']

In [14]:
#fill the table with order's shares
cartesian_df_2 = cartesian_df.merge(orders_2, on='key', how='left')
#drop unnecessary columns
cartesian_df_2 = cartesian_df_2[['Статья FBR', 'Город', 'Месяц', 'Дивизион', 'Макро', 
                                'div_share', 'macro_share', 'total_share']]

In [15]:
#calculate division allocations
allocations_division = cartesian_df_2.merge(cost_division, on=['Месяц', 'Дивизион', 'Статья FBR'], how='left')
allocations_division['allocation_sum'] =  allocations_division['Факт TY'] * allocations_division['div_share']
allocations_division_2 = allocations_division.groupby(['Город', 'Месяц', 'Статья FBR', 'allocation_sum'], 
                                                      as_index=False).agg({'allocation_sum':'sum'}) 
allocations_division_2['Вид расходов'] = 'Аллокации дивизиона' 

#calculate macro allocations
allocations_macro = cartesian_df_2.merge(cost_macro, on=['Месяц', 'Макро', 'Статья FBR'], how='left')
allocations_macro['allocation_sum'] =  allocations_macro['Факт TY'] * allocations_macro['macro_share']
allocations_macro_2 = allocations_macro.groupby(['Город', 'Месяц', 'Статья FBR', 'allocation_sum'], 
                                                      as_index=False).agg({'allocation_sum':'sum'})  
allocations_macro_2['Вид расходов'] = 'Аллокации макро' 

#calculate total allocations
allocations_total = cartesian_df_2.merge(cost_total, on=['Месяц', 'Статья FBR'], how='left')
allocations_total['allocation_sum'] =  allocations_total['Факт TY'] * allocations_total['total_share']
allocations_total_2 = allocations_total.groupby(['Город', 'Месяц', 'Статья FBR', 'allocation_sum'], 
                                                as_index=False).agg({'allocation_sum':'sum'}) 
allocations_total_2['Вид расходов'] = 'Аллокации ТС'

In [16]:
#combine all 3 allocations datasets 
allocations_cons = pd.concat([allocations_division_2, allocations_macro_2, allocations_total_2], ignore_index=True)
allocations_cons = allocations_cons.rename(columns = {'allocation_sum':'Факт TY'})
allocations_cons['СМ/DS'] = 'СМ'
#delete empty rows
allocations_cons = allocations_cons.dropna(subset=['Факт TY'])

In [17]:
#check, difference must be zero
print('Check:', allocations_cons['Факт TY'].sum() - data_ue_pnl_others['Факт TY'].sum())

Check: 0.0


In [18]:
#create df which includes courier cost
data_ue_delivery_cost_2['Статья FBR'] = 'Расходы на доставку'
data_ue_delivery_cost_2['Вид расходов'], data_ue_delivery_cost_2['СМ/DS'] = 'Прямые', 'СМ'

In [19]:
#combine direct stores cost, allocations and darkstores:
ue_final = pd.concat([allocations_cons, data_ue_pnl_stores_2, data_ue_pnl_darkstores_2, 
                      data_ue_delivery_cost_2], ignore_index=True)

In [20]:
#create hierarchy
h = data_ue_pnl[['Город', 'Макро']].drop_duplicates().reset_index(drop=True)
#add macro to the main table
ue_final = ue_final.merge(h, on='Город', how='left')

In [21]:
print('Saving the file...')

Saving the file...


In [22]:
ue_final.to_excel(directory + 'ue_output.xlsx', index=False)